In [3]:
# REQUIREMENTS:
# pip install reportlab qrcode pillow

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.lib import colors
from reportlab.lib.utils import ImageReader
from IPython.display import FileLink, display
import qrcode, os
from datetime import datetime

def render_inventory_form(cfg=None):
    cfg = cfg or {}
    title = cfg.get("title","INVENTORY – ITEMS RECORD")
    # Save explicitly to the current working directory
    filename = cfg.get("filename", os.path.join(os.getcwd(), "egg_inventory_form_a4.pdf"))
    page_size = cfg.get("page_size", A4)
    margins_mm = cfg.get("margins_mm", 6)
    pad_mm = cfg.get("pad_mm", 3)
    label_font = cfg.get("label_font", ("Helvetica","Helvetica-Bold"))
    label_font_sizes = cfg.get("label_font_sizes",(10,7.2))
    value_font_max = cfg.get("value_font_max", 11.5)
    title_font_sizes = cfg.get("title_font_sizes",(18,10))
    base_row_height_mm = cfg.get("row_height_mm", 4.6 * 1.2)
    qr_mm = cfg.get("qr_mm", 16)
    qr_border = cfg.get("qr_border", 1)
    tables_extra_top_gap_mm = cfg.get("tables_extra_top_gap_mm", 6)
    data_rows = cfg.get("rows", [])
    table_rows_override = cfg.get("table_rows", None)
    included_type_labels = cfg.get("included_type_labels","All Types")

    w,h = page_size
    m = margins_mm*mm
    pad = pad_mm*mm
    left,right = m, w-m
    usable_w = right-left
    extra_gap = tables_extra_top_gap_mm*mm

    HEIGHTS = {"single":10*mm,"multi":14*mm}
    FIELDS = [
        ([({"key":"form_name","label":"Form Name"},0.32),
          ({"key":"current_form_qr_code","label":"Form QR Code"},0.23),
          ({"key":"generated_on","label":"Generated On"},0.22),
          ({"key":"generated_by","label":"Generated By"},0.23)],"single"),
        ([({"key":"location","label":"Location"},0.50),
          ({"key":"branch","label":"Branch"},0.50)],"single"),
        ([({"key":"date_iso","label":"Date (YYYY-MM-DD)"},0.50),
          ({"key":"date_name","label":"Date (Name e.g., Oct. 12, 2025)"},0.50)],"single"),
    ]

    def fit_font_size(text,font_name,max_font,min_font,box_width):
        t = "" if text is None else str(text)
        size=max_font
        while size>min_font and pdfmetrics.stringWidth(t,font_name,size)>box_width:
            size-=0.5
        return max(min_font,size)

    def draw_scaled_title(c,text,top_y,side_margin=m,max_font=title_font_sizes[0],min_font=title_font_sizes[1]):
        box_w=w-2*side_margin
        fs=fit_font_size(text,label_font[1],max_font,min_font,box_w)
        c.setFont(label_font[1],fs)
        c.setFillColor(colors.black)
        c.drawCentredString(w/2,top_y,text)
        return fs

    def resolve_height(height_spec,cols):
        if isinstance(height_spec,(int,float)): return float(height_spec)
        if isinstance(height_spec,str) and height_spec in HEIGHTS: return HEIGHTS[height_spec]
        return HEIGHTS["multi"] if len(cols)==1 else HEIGHTS["single"]

    def draw_row(c,top_y,height,cols,prefill):
        c.setLineWidth(1.1)
        c.setStrokeColor(colors.black)
        c.rect(left,top_y-height,usable_w,height,stroke=1,fill=0)
        x=left; acc=0.0
        for i,(field,ratio) in enumerate(cols):
            acc+=ratio
            if i<len(cols)-1:
                x_next=left+usable_w*acc
                c.line(x_next,top_y-height,x_next,top_y)
            col_w=usable_w*ratio
            label=field.get("label","")
            key=field.get("key","")
            fs=fit_font_size(label,label_font[0],label_font_sizes[0],label_font_sizes[1],col_w-2*pad)
            c.setFont(label_font[0],fs); c.setFillColor(colors.black)
            c.drawString(x+pad,top_y-pad-fs*0.2,label)
            val=str(prefill.get(key,""))
            if val:
                vfs=fit_font_size(val,label_font[0],value_font_max,8,col_w-2*pad)
                c.setFont(label_font[1],vfs)
                y_val = top_y - max(6.8*mm, min(height-3.0*mm, height*0.55))
                c.drawString(x+pad,y_val,val)
            x=left+usable_w*acc

    def _make_qr_image(path, value, box_mm=qr_mm, border=qr_border):
        qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=4, border=border)
        qr.add_data(value); qr.make(fit=True)
        img = qr.make_image(fill_color="black", back_color="white")
        img.save(path)
        return box_mm*mm

    def draw_section_caption(c, text, x, y):
        c.setFont(label_font[1], 9.0)
        c.setFillColor(colors.black)
        c.drawString(x, y, text)

    def draw_table(c, x, y_top, col_widths_mm, headers, n_rows, row_h_mm, rows=None, stroke=0.4):
        col_ws = [wmm*mm for wmm in col_widths_mm]
        total_w = sum(col_ws)
        c.setLineWidth(stroke)
        c.setStrokeColor(colors.black)
        c.rect(x, y_top - (row_h_mm*(n_rows+1)), total_w, row_h_mm*(n_rows+1), stroke=1, fill=0)
        cx = x
        for cw in col_ws[:-1]:
            cx += cw
            c.line(cx, y_top - (row_h_mm*(n_rows+1)), cx, y_top)
        for r in range(n_rows+1):
            yy = y_top - (row_h_mm*r)
            c.line(x, yy, x+total_w, yy)
        c.setFont(label_font[1], 7.6)
        cur_x = x
        for i,hdr in enumerate(headers):
            c.drawString(cur_x+1.4*mm, y_top - row_h_mm + 1.8*mm, hdr)
            cur_x += col_ws[i]
        c.setFont("Helvetica", 7.4)
        rows = rows or []
        for r in range(min(n_rows, len(rows))):
            row_vals = ["" if v is None else str(v) for v in rows[r]]
            cur_x = x
            cell_top = y_top - row_h_mm*(r+1)
            for i, cw in enumerate(col_ws):
                text = row_vals[i] if i < len(row_vals) else ""
                avail = cw - 2.2*mm
                fs = fit_font_size(text, "Helvetica", 7.4, 6.0, avail)
                c.setFont("Helvetica", fs)
                c.drawString(cur_x+1.4*mm, cell_top - row_h_mm + 3.2*mm, text)
                cur_x += cw

    def ensure_date_fields(prefill):
        iso = str(prefill.get("date_iso","")).strip()
        name = str(prefill.get("date_name","")).strip()
        if iso and not name:
            try:
                dt = datetime.strptime(iso, "%Y-%m-%d")
                prefill["date_name"] = dt.strftime("%b. %d, %Y")
            except Exception:
                pass
        return prefill

    requested_rows = len(data_rows) + 5
    table_rows = min(15, requested_rows)
    if table_rows_override is not None:
        table_rows = int(table_rows_override)

    c=canvas.Canvas(filename,pagesize=page_size)
    c.setTitle(title)
    prefill = ensure_date_fields(cfg.get("prefill",{}))

    title_y=h-m-5*mm
    draw_scaled_title(c,title,title_y)
    cursor = title_y - 3*mm

    qr_value = str(prefill.get("current_form_qr_code","")).strip()
    qr_path = None
    if qr_value:
        qr_path = "inventory_qr.png"
        qr_box = _make_qr_image(qr_path, qr_value, box_mm=qr_mm, border=qr_border)
        x_qr = (w - qr_box)/2
        y_qr = cursor - qr_box - 1.6*mm
        c.drawImage(qr_path, x_qr, y_qr, width=qr_box, height=qr_box, preserveAspectRatio=True, mask='auto')
        c.setFont(label_font[0],7.2)
        c.drawCentredString(w/2, y_qr - 1.6*mm, qr_value)
        cursor = y_qr - 4.5*mm

    for cols,height_spec in FIELDS:
        h_row=resolve_height(height_spec,cols)
        draw_row(c,cursor,h_row,cols,prefill)
        cursor-=h_row

    cursor -= extra_gap

    headers = ["#","Barcode","Product","Unit","Price","Prev Qty","Add","Sold","Current"]
    col_mm  = [8, 34, 66, 14, 16, 16, 14, 12, 18]
    row_h = base_row_height_mm*mm

    draw_section_caption(c, f"INVENTORY ITEMS – {included_type_labels} ({table_rows} rows)", left, cursor-1.6*mm)
    cursor -= 6*mm
    draw_table(c, left, cursor, col_mm, headers, table_rows, row_h, rows=data_rows)
    cursor -= ((1+table_rows)*row_h + 4*mm)

    bottom_fields = [
        ({"key":"cashier_employee_number","label":"Cashier Employee Number"}, 0.50),
        ({"key":"recorder_employee_number","label":"Recorder Employee Number"}, 0.50),
    ]
    h_row = HEIGHTS["multi"]
    draw_row(c, max(cursor, m+h_row+2*mm), h_row, bottom_fields, prefill)

    c.save()
    try:
        if qr_path and os.path.exists(qr_path): os.remove(qr_path)
    except:
        pass
    try: display(FileLink(filename))
    except: pass
    return os.path.abspath(filename)


# ===== Your provided data.json (unchanged) =====
data_json = {
  "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
  "previous_form_qr_code": "20251011",
  "current_form_qr_code": "20251012",
  "date": "2025-10-12",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "created": { "$date": "2025-10-12T08:00:00Z" },
  "items": [
    {
      "barcode": "4809992000101",
      "type": "fuel",
      "name": "Premium Gasoline",
      "price": 69.50,
      "unit": "liter",
      "previous_quantity": 520,
      "addstock": 300,
      "sold": 250,
      "current_quantity": 570
    },
    {
      "barcode": "4809992000102",
      "type": "fuel",
      "name": "Regular Gasoline",
      "price": 64.75,
      "unit": "liter",
      "previous_quantity": 680,
      "addstock": 400,
      "sold": 320,
      "current_quantity": 760
    },
    {
      "barcode": "4809992000103",
      "type": "fuel",
      "name": "Premium Diesel",
      "price": 61.20,
      "unit": "liter",
      "previous_quantity": 450,
      "addstock": 350,
      "sold": 280,
      "current_quantity": 520
    },
    {
      "barcode": "4809992000201",
      "type": "lubricant",
      "name": "Engine Oil SAE 40",
      "price": 250.00,
      "unit": "liter",
      "previous_quantity": 60,
      "addstock": 40,
      "sold": 25,
      "current_quantity": 75
    },
    {
      "barcode": "4809992000202",
      "type": "lubricant",
      "name": "Automatic Transmission Fluid (ATF)",
      "price": 310.00,
      "unit": "liter",
      "previous_quantity": 50,
      "addstock": 30,
      "sold": 20,
      "current_quantity": 60
    },
    {
      "barcode": "4809992000203",
      "type": "lubricant",
      "name": "Gear Oil EP 90",
      "price": 285.00,
      "unit": "liter",
      "previous_quantity": 45,
      "addstock": 25,
      "sold": 15,
      "current_quantity": 55
    },
    {
      "barcode": "4809992000204",
      "type": "lubricant",
      "name": "Hydraulic Oil ISO 68",
      "price": 295.00,
      "unit": "liter",
      "previous_quantity": 35,
      "addstock": 25,
      "sold": 10,
      "current_quantity": 50
    },
    {
      "barcode": "4809992000205",
      "type": "lubricant",
      "name": "2T Motorcycle Oil",
      "price": 180.00,
      "unit": "liter",
      "previous_quantity": 80,
      "addstock": 60,
      "sold": 45,
      "current_quantity": 95
    }
  ],
  "cashier": "Liam Santos",
  "cashier_employee_number": "50321",
  "recorder": "Ella Reyes",
  "recorder_employee_number": "50325"
}

# ===== Build rows: include BOTH egg and bread (or all types) =====
# Option A (default here): include all types present
items_to_render = data_json["items"]

# If you prefer limiting to certain types, uncomment and edit:
# include_types = {"egg", "bread"}
# items_to_render = [it for it in data_json["items"] if str(it.get("type","")).lower() in {t.lower() for t in include_types}]

rows = []
for i, it in enumerate(items_to_render, start=1):
    rows.append([
        i,
        it.get("barcode",""),
        it.get("name",""),
        it.get("unit",""),
        f'{it.get("price","")}',
        f'{it.get("previous_quantity","")}',
        f'{it.get("addstock","")}',
        f'{it.get("sold","")}',
        f'{it.get("current_quantity","")}'
    ])

# Pad to 15 rows for consistent handwriting space
while len(rows) < 15:
    rows.append([len(rows)+1,"","","","","","","",""])

# Derive included type labels for caption (e.g., "egg, bread")
types_present = sorted({str(it.get("type","")).lower() or "unknown" for it in items_to_render})
included_type_labels = ", ".join(types_present).title()

render_inventory_form({
    "rows": rows,
    "table_rows": 15,
    "included_type_labels": included_type_labels if included_type_labels else "All Types",
    "prefill":{
        "form_name":"Inventory Record",
        "current_form_qr_code":data_json.get("current_form_qr_code",""),
        "generated_on":data_json.get("date",""),
        "generated_by":"JEF Quickeln",
        "location":data_json.get("location",""),
        "branch":data_json.get("branch",""),
        "date_iso":data_json.get("date",""),
        "date_name":"Oct. 12, 2025",
        "cashier_employee_number":data_json.get("cashier_employee_number",""),
        "recorder_employee_number":data_json.get("recorder_employee_number",""),
    },
    "page_size": A4
})


h:\github4\jefstore-gasstations-backend\reportlab\inventories\filled\egg_inventory_form_a4.pdf

'h:\\github4\\jefstore-gasstations-backend\\reportlab\\inventories\\filled\\egg_inventory_form_a4.pdf'